In [ ]:
#to remove the model from my computer when done, it should be stored in ~/.cache/huggingface/transformers/:
rm -rf ~/.cache/huggingface/transformers/facebook/dinov2-large


In [3]:
!pip install torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 1.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.8/150.8 MB 775.0 kB/s eta 0:00:0000:0100:06
  Attempting uninstall: torch
    Found existing installation: torch 2.2.0
    Uninstalling torch-2.2.0:
      Successfully uninstalled torch-2.2.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.17.0 requires torch==2.2.0, but you have torch 2.2.2 which is incompatible.

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [5]:
import torch
from torchvision import transforms
print("Torchvision and transforms imported successfully!")

Torchvision and transforms imported successfully!


In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoImageProcessor, AutoModel
from torchvision import transforms
import numpy as np
import csv
from PIL import Image
import os
from tqdm import tqdm
import time

In [7]:
# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load the DINO model and processor
processor = AutoImageProcessor.from_pretrained("facebook/dinov2-large")
dinov2_model = AutoModel.from_pretrained("facebook/dinov2-large").to(device)

print("DINOv2 model loaded successfully!")

Using device: cpu


config.json: 100%|█████████████████████████████| 549/549 [00:00<00:00, 1.41MB/s]
model.safetensors: 100%|███████████████████| 1.22G/1.22G [01:07<00:00, 18.0MB/s]


DINOv2 model loaded successfully!


In [8]:
# Set the model to evaluation mode
dinov2_model.eval()

# Path to your image folder
#path_to_data_folder = "/content/drive/MyDrive/w281_Final_Project_Leaf_Disease_Classification_Computer_Vision/Data/train_images"
path_to_data_folder = "/Users/tamarbrand-perez/Library/CloudStorage/GoogleDrive-terez@berkeley.edu/My Drive/w281_Final_Project_Leaf_Disease_Classification_Computer_Vision/Data/train_images"

# Batch size
BATCH_SIZE = 32

# Function to process a batch of images and get their features
def get_batch_features(image_paths):
    # Measure image loading time
    load_start = time.time()
    images = [Image.open(path).convert('RGB') for path in image_paths]
    load_end = time.time()
    load_time = load_end - load_start
    print(f"Loaded {len(image_paths)} images in {load_time:.2f} seconds")

    # Measure preprocessing and inference time
    infer_start = time.time()
    inputs = processor(images=images, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = dinov2_model(**inputs)
    infer_end = time.time()
    infer_time = infer_end - infer_start
    print(f"Preprocessed and inferred {len(image_paths)} images in {infer_time:.2f} seconds")
    print(outputs[0][:, 0, :].shape)
    return outputs[0][:, 0, :]

# Get list of image files
image_files = [f for f in os.listdir(path_to_data_folder)
               if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

# Process images in batches
all_features = []
image_names = []

total_start = time.time()

for i in tqdm(range(0, len(image_files), BATCH_SIZE)):
    batch_files = image_files[i:i+BATCH_SIZE]
    batch_paths = [os.path.join(path_to_data_folder, f) for f in batch_files]

    batch_features = get_batch_features(batch_paths)

    all_features.extend(batch_features.cpu().numpy())
    image_names.extend(batch_files)

total_end = time.time()
total_time = total_end - total_start

# Convert to numpy array
all_features = np.array(all_features)

# Save features to CSV
output_csv_path = "dino_features_test.csv"
with open(output_csv_path, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    # Write header
    writer.writerow(['image_name'] + [f'feature_{i}' for i in range(all_features.shape[1])])
    # Write data
    for name, features in zip(image_names, all_features):
        writer.writerow([name] + features.tolist())

print(f"Features saved to {output_csv_path}")
print(f"Processed {len(image_names)} images")
print(f"Total processing time: {total_time:.2f} seconds")

  0%|                                                   | 0/562 [00:00<?, ?it/s]

Loaded 32 images in 18.81 seconds


  0%|                                        | 1/562 [01:19<12:22:57, 79.46s/it]

Preprocessed and inferred 32 images in 60.63 seconds
torch.Size([32, 1024])
Loaded 32 images in 21.71 seconds


  0%|▏                                       | 2/562 [02:38<12:17:36, 79.03s/it]

...

Preprocessed and inferred 32 images in 54.27 seconds
torch.Size([32, 1024])
Loaded 32 images in 16.59 seconds


 36%|█████████████▍                       | 204/562 [4:13:26<7:05:42, 71.35s/it]

Preprocessed and inferred 32 images in 54.06 seconds
torch.Size([32, 1024])
Loaded 32 images in 17.11 seconds


 36%|█████████████▍                       | 204/562 [4:13:50<7:25:27, 74.66s/it]


KeyboardInterrupt: 

In [9]:
import os
print("Current working directory:", os.getcwd())

Current working directory: /Users/tamarbrand-perez/Desktop/Berkeley/w281 computer vision/final project/w281_final_project_plants
